# Introduction to LangChain.js

Welcome! This short course will introduce you to [LangChain.js](https://github.com/langchain-ai/langchainjs), a framework for building large language model (LLM) powered, context-aware, reasoning applications. By the end of the course, you'll learn all the concepts you need to create your own version of a popular example of this type of app: a "chat with data" chain that lets you ask questions about a document's contents using natural language.

This course follows on from the previous DeepLearning.ai courses on the Python version of LangChain. Much like LangChain.js itself, it's intended for web developers and others in the broader JavaScript ecosystem interested in building with LLMs, and will dive more deeply into features like streaming and integration with standard web/JavaScript APIs.

**Note:** This notebook uses the [Deno](https://deno.com/) Jupyter kernel, and has slightly different import statements from those you may be familiar with in Node and web runtimes. You can convert them to Node/web imports by swapping `npm:langchain@0.0.178` for `langchain`. To run these course notebooks locally, see [these setup instructions](https://docs.deno.com/runtime/manual/tools/jupyter).

**Note:** Throughout this course, we'll link to explorable traces in [LangSmith](https://smith.langchain.com/), LangChain's accompanying observability platform, that illustrate how the different example chains work. If you're following along locally, you'll need to set a few environment variables [as documented here](https://docs.smith.langchain.com/).

## Building Blocks: LangChain Expression Language

At the core of LangChain is LangChain Expression Language (LCEL). LCEL is a composable syntax for chaining LangChain modules together. Objects that are compatible with LCEL are called `Runnables`.

To show how this works, we'll compose a simple chain containing two of LangChain's primary modules: a prompt template that formats our input and an OpenAI chat model:

In [1]:
import { ChatPromptTemplate } from "npm:langchain@0.0.178/prompts";
import { ChatOpenAI } from "npm:langchain@0.0.178/chat_models/openai";

Deno.env.set("OPENAI_API_KEY", "");
Deno.env.set("LANGCHAIN_TRACING_V2", "true");
Deno.env.set("LANGCHAIN_SESSION", "");
Deno.env.set("LANGCHAIN_API_KEY", "");

const prompt = ChatPromptTemplate.fromTemplate(`What are three good names for a company that makes {product}?`);

const model = new ChatOpenAI({});

const chain = prompt.pipe(model);

The prompt template will take input variables and inject them into the template. The chain then pipes the properly formatted result into the chat model as input. We can run the LCEL chain as follows:

In [2]:
await chain.invoke({
  product: "colorful socks",
});

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "1. ChromaSock\n2. VividThread\n3. KaleidoSock",
    additional_kwargs: { function_call: undefined }
  },
  lc_namespace: [ "langchain", "schema" ],
  content: "1. ChromaSock\n2. VividThread\n3. KaleidoSock",
  name: undefined,
  additional_kwargs: { function_call: undefined }
}

And we can see that the result is an `AIMessage` with three good names for a company that makes colorful socks.

If we just want the output as a raw string rather than an AI message response, we can add a third step to our chain: an output parser!

In [3]:
import { StringOutputParser } from "npm:langchain@0.0.178/schema/output_parser";

const outputParser = new StringOutputParser();

const nameGenerationChain = prompt.pipe(model).pipe(outputParser);

await nameGenerationChain.invoke({
  product: "fancy cookies",
});

"1. Delicieux Delights\n2. Gourmet Galettes\n3. Decadent Crumbs"

You can see a LangSmith trace of the above example [here](https://smith.langchain.com/public/02c0469b-f3b4-4681-b006-da74df897dfa/r).

These three pieces form the core of many more complicated chains.

## Streaming

One of the many advantages to LCEL is that chains composed in this fashion get certain methods automatically. One useful one is `.stream()`, which returns output from the chain in an iterable stream. Because LLM responses often take a long time to finish, streaming is useful in situations where showing feedback quickly is important.

Here's an example with the chain we just composed:

In [4]:
const stream = await nameGenerationChain.stream({
  product: "really cool robots",
});

for await (const chunk of stream) {
  console.log(chunk);
}


1
.
 Rob
o
Tech


2
.
 Mech
Wave


3
.
 Fut
ura
B
ots



Above, the `model` emits partial message chunks, and the output parser transforms those streamed chunks from the model as they are generated, resulting in immediate string output.

This stream is a special `ReadableStream` that implements the iterator interface and can be passed directly to e.g. a `fetch` built-in `Response` object and returned in popular web frameworks like Next.js.

In [5]:
const stream = await nameGenerationChain.stream({
  product: "superb owls",
});

new Response(stream);

Response {
  body: IterableReadableStream { locked: false },
  bodyUsed: false,
  headers: Headers {},
  ok: true,
  redirected: false,
  status: 200,
  statusText: "",
  url: ""
}

## Chaining with `RunnableMap`

While sequentially chaining calls together like this can be useful on its own, more complex chains often need to combine inputs from different sources and steps. For example, let's say we want to take the company name outputs generated by the above chain and pass it as one input of several into another chain that picks the best one. One way to do this is to add a `RunnableMap` to the sequence. We also declare the sequence using the alternate `RunnableSequence` factory method for readability:

In [6]:
import { RunnableSequence, RunnableMap } from "npm:langchain@0.0.178/schema/runnable";

const finalPrompt = ChatPromptTemplate.fromTemplate(`Pick the best name from {names} based on {final_criteria} and explain why.`);

const combinedChain = RunnableSequence.from([
  RunnableMap.from({
    names: nameGenerationChain,
    final_criteria: (input) => input.final_criteria,
  }),
  finalPrompt,
  model,
  new StringOutputParser(),
]);

Each property in the `RunnableMap` gets called in parallel with the input from the previous step. In the above example, since the `RunnableMap` is the first step, it is called with the input to the overall chain. The output of the `RunnableMap` is an object with the result of that call. Since we want to pass `final_criteria` through to `finalPrompt`, we simply extract the property from the input.

We can see the result below:

In [7]:
await combinedChain.invoke({
  product: "wooden cars",
  final_criteria: "sustainability",
});

"From a sustainability perspective, the best name would be WoodWheelz.\n" +
  "\n" +
  "WoodWheelz implies the use of"... 598 more characters

This [LangSmith trace](https://smith.langchain.com/public/ed4d767a-49d5-427c-97fe-fc2dc3f5aba5/r) shows what's going on under the hood - the `nameGenerationChain` we defined above generated three good names for a company that makes wooden cars, and the output was passed to our `finalPrompt` along with the `final_criteria` we originally passed to make a final judgement of the best name.